In [6]:
import pandas as pd
import numpy as np

# META

# Introduction
This file is used to wrangle the permit class types

## Conclusion
Was able to grab all the unique permit types and their respective codes. Exported them to a .csv under `data\dict\`. Left a function that can be used to see all the descriptions that are available for each permit code


# Gather and load data
I will be using the built-in query service on Open Data Calgary.

Query description:
- Kept rows `PermitNum`, `AppliedDate`, `PermitType`, `PermitTypeMapped`, `PermitClass`, `PermitClassGroup`, `PermitClassMapped`, `Description`
- Sorted by `AppliedDate` ascending
- Exported as `permit_classes.csv`

In [22]:
# Load permit_classes.csv
source_df = pd.read_csv('data/raw/permit_classes.csv')
source_df.head()

,PermitNum,AppliedDate,PermitType,PermitTypeMapped,PermitClass,PermitClassGroup,PermitClassMapped,Description
0,BP1999-07500,1999/06/22,Single Construction Permit,Building,1106 - Single Family House,Single Family,Residential,"SFD,FIREPLACE,GARAGE,"
1,BP1999-07510,1999/06/23,Commercial / Multi Family Project,Building,4002 - Vacant,General,Non-Residential,THIS IS A GARBAGE PERMIT - jdh 99/07/02
2,BP1999-07520,1999/06/23,Single Construction Permit,Building,1106 - Single Family House,Single Family,Residential,"SFD,FIREPLACE,GARAGE,PORCH,"
3,BP1999-07558,1999/06/23,Residential Improvement Project,Building,1104 - Deck,Single Family,Residential,Deck
4,BP1999-07542,1999/06/23,Single Construction Permit,Building,1106 - Single Family House,Single Family,Residential,"SFD,FIREPLACE,GARAGE,"


In [65]:
# Get unique PermitClass code and put them into a list
unique_permitclass = source_df['PermitClass'].unique()
unique_permitclass.shape

(324,)

Now, I want a dictionary that shows me the permit code and what it means.
To do this, I first want to separate the permit code and the description that is alongside it

In [41]:
permit_dict_list = []
for permitclass in unique_permitclass :
    permit_dict_list.append(str(permitclass).split(' - '))
permit_dict_df = pd.DataFrame(permit_dict_list)
print(permit_dict_df)

         0                             1     2
0     1106           Single Family House  None
1     4002                        Vacant  None
2     1104                          Deck  None
3     1301       Private Detached Garage  None
4     3605                    Food Kiosk  None
..     ...                           ...   ...
319   1111             Attached At Grade  None
320   2502             Experimental Farm  None
321  `1101  Improvements (Single Family)  None
322   2307                  Railway Shop  None
323   3202                Grain Elevator  None

[324 rows x 3 columns]


So, we have an extra column that I was not expecting. Lets find those elements that are `None`. My guess is that it is something irrelevant but we should make sure

In [50]:
print(permit_dict_df.dropna())

        0                      1                                    2
81   3609             Restaurant                  Take-Out (No Seats)
258  3613             Restaurant                             Licensed
263  3614             Restaurant  Licensed And Drinking Establishment
279  7102  Special Function Tent                         Recreational
281  7101  Special Function Tent                           Commercial
283  7103       Special Function                              Class 1
296  7104       Special Function                              Class 2


Okay! We are good! Lets continue

First, convert all the codes to `int` and compress the last two columns. We are going to do this by going backwards and recreating our `permit_dict_df`

In [105]:
permit_dict_list = []
for permitclass in unique_permitclass :
    insertion = str(permitclass).split(' - ')

    # compressing the last two columns
    if( len(insertion) > 2 ) :
        desc = ' - '.join(insertion[1:])
        insertion = [insertion[0], desc]

    # converting to int
    try : 
        insertion[0] = int(insertion[0])
    except :
        if insertion[0] == 'nan' : # I've had this rogue 'nan' at 271 that has no information.
            continue
        insertion[0] = int(''.join(digit for digit in insertion[0] if digit.isdigit()))
    
    permit_dict_list.append(insertion)

permit_dict_df = pd.DataFrame(permit_dict_list, columns=['permit_code','permit_desc'])
print(permit_dict_df)

     permit_code                   permit_desc
0           1106           Single Family House
1           4002                        Vacant
2           1104                          Deck
3           1301       Private Detached Garage
4           3605                    Food Kiosk
..           ...                           ...
318         1111             Attached At Grade
319         2502             Experimental Farm
320         1101  Improvements (Single Family)
321         2307                  Railway Shop
322         3202                Grain Elevator

[323 rows x 2 columns]


Now we can check how many unique codes there actually are

In [102]:
unique_codes = permit_dict_df['permit_code'].unique()
print(len(unique_codes))

264


A-ha! So there are actually only 264 unique codes! I want to remove the duplicates and sort it by `permit_code`

In [106]:
permit_dict_df_slim = permit_dict_df.drop_duplicates(subset='permit_code', ignore_index=True).copy()
permit_dict_df_slim.sort_values(by='permit_code', inplace=True)
print(permit_dict_df_slim)

     permit_code                           permit_desc
5           1101                  Basement Development
7           1102                               Carport
25          1103                  Solid Fuel Fireplace
2           1104                                  Deck
52          1105                        Retaining Wall
..           ...                                   ...
235         7101    Special Function Tent - Commercial
233         7102  Special Function Tent - Recreational
237         7103            Special Function - Class 1
246         7104            Special Function - Class 2
124         9999                           Unspecified

[264 rows x 2 columns]


Now, some of the `permit_desc` will be too narrow of a description and maybe not encapsulate what the permit codes actually means.

So, I will create a function that will print all the duplicates from the original `permit_dict_df`

If I want to return to this and not have to run all of this stuff. I should export both of the `DataFrame`s and then import them when defining the function.

In [109]:
permit_dict_df.to_csv('data/dict/permit_class.csv')
permit_dict_df_slim.to_csv('data/dict/permit_class_slim.csv')

# Code to return to

In [125]:
permit_dict_df = pd.read_csv('data/dict/permit_class.csv')
permit_dict_df_slim = pd.read_csv('data/dict/permit_class_slim.csv')

def printPermitCodeDescriptions(code) :
    print(permit_dict_df.loc[permit_dict_df['permit_code'] == code])

In [131]:
printPermitCodeDescriptions(1106)

    Unnamed: 0  permit_code          permit_desc
0            0         1106  Single Family House
9            9         1106             Addition
34          34         1106                House
